In [ ]:
!pip install --upgrade gensim
!pip install pymorphy2

In [ ]:
import pandas as pd
from tqdm import tqdm

from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize
import nltk
import re
import pymorphy2
import numpy as np

Теперь загрузим модель, обученную разработчиками проекта RusVectores для русского языка на новостях. В зависимости от того, откуда вы берете модели, они могут загружаться по-разному. Более того, модель можно обучить самому - для этого нужно просто взять много размеченных текстов.

In [3]:
model_w2v = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Machine Learning (ITHUB) /ДИ 2023/lessons/data/araneum_upos_skipgram_300_2_2018.vec.gz')

Теперь можно получить представление слов в новом пространстве. Имейте в виду, что в данной модели они идут с частями речи!

In [4]:
model_w2v['программирование_NOUN']

array([ 0.049955,  0.057788, -0.0008  , -0.045055, -0.015318, -0.006786,
       -0.031389, -0.057646,  0.044001,  0.020228, -0.066561,  0.069199,
        0.006213, -0.012925, -0.023736, -0.028251, -0.032314,  0.076892,
        0.0305  ,  0.024869,  0.071448, -0.126915,  0.048925,  0.026423,
       -0.041038, -0.014302, -0.01585 ,  0.01019 , -0.009125, -0.038794,
       -0.065042,  0.060425, -0.053604, -0.11098 ,  0.077359, -0.007369,
        0.030728,  0.074339,  0.06024 , -0.09928 ,  0.143421, -0.047111,
       -0.083323, -0.061702, -0.013478,  0.071922, -0.014687,  0.003868,
       -0.040532,  0.003834,  0.041084,  0.072017, -0.016087,  0.10976 ,
       -0.024698,  0.101379, -0.047046, -0.006745,  0.097262,  0.056536,
       -0.000801, -0.071755, -0.007446,  0.004823,  0.062799,  0.0298  ,
       -0.033715, -0.0595  ,  0.039364, -0.130194,  0.043577,  0.024074,
        0.006667,  0.032938,  0.060546,  0.08892 ,  0.073503,  0.056737,
       -0.067149,  0.036383, -0.021797, -0.035148, 

In [5]:
model_w2v.most_similar('программирование_NOUN')

[('програмирование_NOUN', 0.8259077072143555),
 ('объектно-ориентированный_ADJ', 0.7065892219543457),
 ('ассемблер_NOUN', 0.6736835241317749),
 ('алгоритмизация_NOUN', 0.6538978815078735),
 ('python_X', 0.6437264084815979),
 ('pascal_X', 0.6302998065948486),
 ('программировать_VERB', 0.6270321011543274),
 ('actionscript_X', 0.6192384958267212),
 ('perl_X', 0.6156083941459656),
 ('assembler_X', 0.6134674549102783)]

In [6]:
model_w2v.most_similar('вакансия_NOUN')

[('резюме_NOUN', 0.6046861410140991),
 ('вакантный_ADJ', 0.5599281787872314),
 ('соискатель_NOUN', 0.555031418800354),
 ('должность_NOUN', 0.542113184928894),
 ('трудоустраиваться_VERB', 0.5227364897727966),
 ('офис-менеджер_NOUN', 0.5094016194343567),
 ('трудоустраивать_VERB', 0.490702360868454),
 ('трудоустройство_NOUN', 0.48329246044158936),
 ('рекрутер_NOUN', 0.4818519949913025),
 ('разнорабочий_NOUN', 0.47602003812789917)]

Возможны векторные операции

In [8]:
model_w2v.most_similar(positive=[u'король_NOUN', u'женщина_NOUN'],negative=[u'мужчина_NOUN'])

[('королева_NOUN', 0.7022064328193665),
 ('герцог_NOUN', 0.6763379573822021),
 ('монарх_NOUN', 0.6703874468803406),
 ('генрих_PROPN', 0.656510055065155),
 ('людовик_PROPN', 0.656411349773407),
 ('франция::людовик_PROPN', 0.6421706080436707),
 ('принц_NOUN', 0.6407060027122498),
 ('королевство_NOUN', 0.6257355809211731),
 ('наследный_ADJ', 0.6177536845207214),
 ('герцогиня_NOUN', 0.617081344127655)]

Можно найти лишнее слово в последовательности.

In [9]:
model_w2v.doesnt_match(['программирование_NOUN', 'инженер_NOUN', 'аналитик_NOUN', 'кривляние_NOUN'])

'кривляние_NOUN'

Беда Word2Vec состоит в том, что он генерирует только один вектор на каждое слово. То есть он не различает различных смыслов одного слова.

In [10]:
df = pd.read_csv('/content/drive/MyDrive/Machine Learning (ITHUB) /ДИ 2023/lessons/data/zayavki_text.csv')
df.head()

,text,category,id_category,norm_text
0,"1й подъезд, 5 этаж. В коридоре над висит ка...",Содержание МКД,10,й подъезд этаж в коридор над висеть какойтый п...
1,Маленькмй лифт изрисован внутри.,Содержание МКД,10,маленькмть лифт изрисовать внутри
2,реклама на светофоре,Благоустройство,3,реклама на светофор
3,"1й подъезд, 5 этаж, квартирный холл. Рааботник...",Содержание МКД,10,й подъезд этаж квартирный холл рааботник тсж л...
4,незаконная решетка на фасаде,Нарушение правил пользования общим имуществом,23,незаконный решётка на фасад


In [11]:
conv_pos = {'ADJF':'ADJ', 'ADJS':'ADJ', 'ADV':'ADV', 'NOUN':'NOUN',
            'VERB':'VERB', 'PRTF':'ADJ', 'PRTS':'ADJ', 'GRND':'VERB'}

tmp_dict = {} # Кеш значимых слов.
nones = {} # Кеш незначимых слов.

morph = pymorphy2.MorphAnalyzer()

# Фильтруем по части речи и возвращаем только начальную форму.
def normalizePymorphy(text, need_pos=True):
    tokens = re.findall('[A-Za-zА-Яа-яЁё]+\-[A-Za-zА-Яа-яЁё]+|[A-Za-zА-Яа-яЁё]+', text)
    words = []
    for t in tokens:
        # Если токен уже был закеширован, быстро возьмем результат из него.
        if t in tmp_dict.keys():
            words.append(tmp_dict[t])
        # Аналогично, если он в кеше незначимых слов.
        elif t in nones.keys():
            pass
        # Слово еще не встретилось, будем проводить медленный морфологический анализ.
        else:
            pv = morph.parse(t)
            if pv[0].tag.POS != None:
                if pv[0].tag.POS in conv_pos.keys():
                    if need_pos:
                        word = pv[0].normal_form+"_"+conv_pos[pv[0].tag.POS]
                    else:
                        word = pv[0].normal_form
                    # Отправляем слово в результат, ...
                    words.append(word)
                    # ... и кешируем результат его разбора.
                    tmp_dict[t] = word
                else:
                    # Для незначимых слов можно даже ничего не хранить. Лишь бы потом не обращаться к морфологии.
                    nones[t] = ""

    return words

In [12]:
sentences = []
for text in tqdm(df['norm_text']):
    sentences.append(normalizePymorphy(text))

100%|██████████| 59889/59889 [00:06<00:00, 9896.71it/s] 


In [13]:
len(sentences)

59889

In [14]:
num_features = 100  # итоговая размерность вектора каждого слова
min_word_count = 5  # минимальная частотность слова, чтобы оно попало в модель
num_workers = 2    # количество ядер вашего процессора, чтоб запустить обучение в несколько потоков
context = 10        # размер окна
downsampling = 1e-3 # внутренняя метрика модели

model = Word2Vec(sentences, workers=num_workers, vector_size=num_features,
                 min_count=min_word_count, window=context, sample=downsampling)

In [15]:
model.init_sims(replace=True)

<ipython-input-15-dabd6e7393f0>:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


In [16]:
model.wv['фонарь_NOUN']

array([ 0.06487446,  0.09039703, -0.0089508 ,  0.02401686,  0.15718935,
       -0.17154811,  0.05089436,  0.14242958, -0.12028002,  0.19513063,
       -0.16639969, -0.1265041 ,  0.1378115 , -0.03615012, -0.03115026,
        0.07448308, -0.09634089,  0.01913547,  0.00590958, -0.01881026,
        0.03835947, -0.09388053,  0.01846501, -0.08913436,  0.10956341,
       -0.11699503, -0.12190484, -0.09884856, -0.18817177, -0.02336179,
        0.0042353 ,  0.15941611,  0.00045534,  0.02824051,  0.01739237,
        0.07928156, -0.17782359,  0.01256612,  0.02339344, -0.05683424,
        0.14706391, -0.06824049,  0.07237076,  0.12325244,  0.05166738,
        0.10029148, -0.03752097,  0.07082177,  0.01920119, -0.19724087,
        0.03545635,  0.13333532,  0.0052082 , -0.08736414,  0.14399673,
       -0.09812599,  0.05510534, -0.04188606,  0.15268368, -0.0199583 ,
       -0.00148719, -0.04491752, -0.0991875 ,  0.04725712, -0.00927151,
        0.01378785,  0.18398863,  0.01075747,  0.02826921, -0.05

In [24]:
model.wv.most_similar('хабарик_NOUN')

[('обломок_NOUN', 0.9560327529907227),
 ('пивная_NOUN', 0.9528722763061523),
 ('матрас_NOUN', 0.9468836188316345),
 ('пачка_NOUN', 0.9394866824150085),
 ('обилие_NOUN', 0.9381367564201355),
 ('обёртка_NOUN', 0.9371911287307739),
 ('маска_NOUN', 0.9324549436569214),
 ('пенопласт_NOUN', 0.9289234280586243),
 ('стаканчик_NOUN', 0.9262374043464661),
 ('завал_NOUN', 0.9261230230331421)]

In [25]:
model.corpus_total_words

406062

У нас есть смысл отдельных слов. Построим на его основе смысл текста как среднее арифметическое всех векторов для слов, составляющих данный текст.

In [26]:
index2word_set = set(model_w2v.index_to_key)

def text_to_vec(text):
    text_vec = np.zeros((model_w2v.vector_size,), dtype="float32")
    n_words = 0

    for word in normalizePymorphy(text):
        if word in index2word_set:
            n_words = n_words + 1
            text_vec = np.add(text_vec, model_w2v[word])

    if n_words != 0:
        text_vec /= n_words
    return text_vec

In [27]:
w2v_vectors = [text_to_vec(text) for text in df['norm_text']]

In [28]:
len(w2v_vectors)

59889

In [30]:
w2v_vectors[0]

array([ 2.38467269e-02,  4.30729128e-02, -3.32309026e-03, -1.53742731e-02,
        5.39381849e-03, -3.06300051e-03, -2.86790021e-02,  1.32808182e-02,
        5.42049073e-02,  1.09496387e-02, -1.35353645e-02,  1.63030028e-02,
        2.54950896e-02,  5.17941862e-02, -4.02886346e-02, -2.33333632e-02,
        4.98680919e-02,  5.68403602e-02, -9.09227226e-03, -5.23545481e-02,
       -2.11916380e-02, -1.45475455e-02, -5.09700039e-03,  1.30629996e-02,
       -4.73375469e-02,  2.80328169e-02,  7.07247257e-02, -8.12182650e-02,
        3.14929076e-02, -2.99610011e-02,  1.58829093e-02,  1.77835450e-02,
        2.96129100e-02, -3.14676352e-02, -7.89045449e-03,  1.01585453e-02,
       -1.02390908e-02, -5.06618107e-03, -1.85106378e-02, -4.78008240e-02,
        1.82774533e-02,  2.13951822e-02,  6.51554484e-03, -4.93411794e-02,
       -9.74127278e-03,  3.43321823e-02, -2.89656352e-02,  1.53318199e-03,
       -4.00227215e-03,  2.83381809e-03,  3.06321811e-02, -4.09609126e-03,
       -1.45252738e-02,  

Найдем текст, самый похожий на некоторый текст.

In [36]:
for index,text in enumerate(df['norm_text']):
  if 'хабарик' in text:
    print(index,text)

755 почти всё почтовый ящик в наш подъезд сломать и не закрываться уже пара год сначала мы думаличтый ктоть это исправить но нет потом привыкнуть больший проблема это не вызывать но сейчас приехать много новый людейктоть снимать в ящик кидать мусор хабарик пропадать квитанция и корреспонденция сам починить ящик я не мочь поэтому просить помочь разобраться в это вопрос ящик сломать многие ноести нужный конкретикаменить волновать ящик квартира фото нечёткий ткно лестница довольно темноно думаюэтый не так важно спасибо
2605 этот кучка гнилой лист ещё долго тут лежать быть и догнивать там уже свалка образоваться изз недалёкий человек думать что туда можно кидать свой мусор бутылка также валяться рядом с это безобразие или вы хотеть чтобы ктоть кинуть туда хабарик непотушенный и всё это загореться
21962 на лестничный пролёт го подъезд не проводиться уборка хабарик валяться на пол четвёртый день
23142 хабарик на газон
26350 не убрать около окно между и этаж подьезда валяться хабарик сигарета

In [38]:
vpos = 755
v1 = text_to_vec(df['norm_text'][vpos])
print(df['norm_text'][vpos])

ms = np.argmin([np.linalg.norm(v1-w2v_vectors[i]) for i in range(df['norm_text'].shape[0]) if i!=vpos])
#print(df['norm_text'][vpos])
if ms>vpos:
    print(df['norm_text'][ms+1])
else:
    print(df['norm_text'][ms])

почти всё почтовый ящик в наш подъезд сломать и не закрываться уже пара год сначала мы думаличтый ктоть это исправить но нет потом привыкнуть больший проблема это не вызывать но сейчас приехать много новый людейктоть снимать в ящик кидать мусор хабарик пропадать квитанция и корреспонденция сам починить ящик я не мочь поэтому просить помочь разобраться в это вопрос ящик сломать многие ноести нужный конкретикаменить волновать ящик квартира фото нечёткий ткно лестница довольно темноно думаюэтый не так важно спасибо
просьба в парадный номер повесить на первый этаж ящик для сбор не нужный корреспонденция реклама возле почтовый ящик образец приложить на второй фото


Сожмем вектора, которые мы получили - попробуем визуализировать

In [39]:
!pip install umap-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.4-py3-none-any.whl size=86770 sha256=4b9f328f229348a6065d87d33db2957b84579fd9091178e24fe9c553cc81871b
  Stored in directory: /root/.cache/pip/wheels/fb/66/29/199acf5784d0f7b8add6d466175ab45506c96e386ed5dd0633
  Created wheel for pynndescent: filename=pynndescent-0.5.10-py3-none-any.whl size=55615 sha256=9f1a1f0e968484bef337c1e30c2552244736c319a160630591fdd55d36fd6366
  Stored in directory: /root/.cache/pip/wheels/4a/38/5d/f60a40a66a9512b7e5e83517ebc2d1b42d857be97d135f1096
Successfully built umap-learn pynndescent


In [40]:
import plotly.graph_objs as go
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap

In [41]:
def show_fig(new_shape):
    fig = go.Figure(go.Scatter3d(x=new_shape[:, 0],
                        y=new_shape[:, 1],
                        z=new_shape[:, 2],
                        marker=dict(opacity=0.9,
                                    reversescale=True,
                                    colorscale='Blues',
                                    size=5,
                                    color=df['id_category']),
                        line=dict (width=0.02),
                        mode='markers',
                                 text = df['category']))
    fig.update_layout(
        scene = dict(
            xaxis = dict(nticks=4),
                         yaxis = dict(nticks=4),
                         zaxis = dict(nticks=4),),
        width=700,
        margin=dict(r=20, l=10, b=10, t=10))
    fig.show()

In [42]:
pca=PCA(n_components=3)
reduced_pca = pca.fit_transform(w2v_vectors[:1000])
show_fig(reduced_pca)

Output hidden; open in https://colab.research.google.com to view.

In [45]:
type(w2v_vectors[:1000])

list

In [46]:
tsne=TSNE(n_components=3)
reduced_tsne = tsne.fit_transform(np.array(w2v_vectors[:1000]))
show_fig(reduced_tsne)

Output hidden; open in https://colab.research.google.com to view.

In [44]:
umap_emb = umap.UMAP(n_components=3)
reduced_umap = umap_emb.fit_transform(w2v_vectors[:1000])
show_fig(reduced_umap)

Output hidden; open in https://colab.research.google.com to view.